# Health Data - GTT Testing after 24 Weeks

#### Import libraries and Data, Drop unique identifier 'MRN', sort by ID, rename column, export csv and reimport cleaned csv

In [1]:
import os
import numpy as np
import pandas as pd
import sklearn
from scipy import stats
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

data_1 = "~/Downloads/health_2.csv"
df = pd.read_csv(data_1)

df = df.drop(columns = "MRN", axis = 1)

df = df.rename(columns={"Auto Before_After": "Before_After"})
df = df.sort_values(by ='ID')


compression_opts = dict(method='zip',
                        archive_name='Health.csv')

df.to_csv('Health.zip', index=False,
          compression=compression_opts)

data_2 = "~/Desktop/portfolio/Portfolio/06_Health/Health.csv"
df = pd.read_csv(data_2)

In [2]:
df.head()

,ID,LANGUAGE,AGE,GTT_TEST_RESULT,High_WNL,Before_After,LMP_DATE,24w_DATE,GTT_TEST_NAME,GTT_RESULT_DATE
0,1,Spanish,31,157,High,After,6/4/2019,11/19/2019,"GLUCOSE, GESTATIONAL SCREEN (50G)",2/18/2020
1,1,Spanish,31,73,WNL,After,6/4/2019,11/19/2019,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",2/22/2020
2,1,Spanish,31,163,WNL,After,6/4/2019,11/19/2019,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",2/22/2020
3,1,Spanish,31,117,WNL,After,6/4/2019,11/19/2019,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",2/22/2020
4,1,Spanish,31,131,WNL,After,6/4/2019,11/19/2019,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",2/22/2020


#### Check for any existing NULL values

In [3]:
df.isnull().sum()

ID                 0
LANGUAGE           0
AGE                0
GTT_TEST_RESULT    0
High_WNL           0
Before_After       0
LMP_DATE           0
24w_DATE           0
GTT_TEST_NAME      0
GTT_RESULT_DATE    0
dtype: int64

#### Total Number of observations in Data Set

In [4]:
df['ID'].count()

9708

In [5]:
df.head(5)

,ID,LANGUAGE,AGE,GTT_TEST_RESULT,High_WNL,Before_After,LMP_DATE,24w_DATE,GTT_TEST_NAME,GTT_RESULT_DATE
0,1,Spanish,31,157,High,After,6/4/2019,11/19/2019,"GLUCOSE, GESTATIONAL SCREEN (50G)",2/18/2020
1,1,Spanish,31,73,WNL,After,6/4/2019,11/19/2019,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",2/22/2020
2,1,Spanish,31,163,WNL,After,6/4/2019,11/19/2019,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",2/22/2020
3,1,Spanish,31,117,WNL,After,6/4/2019,11/19/2019,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",2/22/2020
4,1,Spanish,31,131,WNL,After,6/4/2019,11/19/2019,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",2/22/2020


### Create array with patient ID and count the number of unique ID's


In [6]:

count = 0
arr = []
arr_temp = []

for i in range(0,len(df['ID'])):
    if df['ID'][i] not in arr_temp:
        arr_temp.append(df['ID'][i])
        arr.append(f"Patient_{df['ID'][i]}")
        count += 1
               
print(f"There are {count} unique patients in the dataset")
# print(len(arr_temp))
# print(len(arr))


There are 1505 unique patients in the dataset


#### Create two copies of dataset, one filtering for data "Before 24 weeks" and one for data "After 24 weeks"

In [7]:
df_before = df[df['Before_After'] == 'Before']
print(len(df_before))

df_after = df[df['Before_After'] == 'After']
print(len(df_after))


5156
4552


#### Create Dictionary with Key = Patient ID and Value = age, average test result, etc before/after treatment


In [8]:

before_dict = {}

for i in range(1,1505):
    before_dict[arr[i]] = [round(df_before['GTT_TEST_RESULT'][df_before['ID'] == arr_temp[i]].mean(),2), df_before['AGE'][df_before["ID"] == arr_temp[i]].mean(), df_before['ID'][df_before["ID"] == arr_temp[i]].mean()]
    
print(f"There are {len(before_dict)} observations in the 'before' dataset")


df_after = df[df['Before_After'] == 'After']
after_dict = {}

for i in range(1,1505):
    after_dict[arr[i-1]] = [round(df_after['GTT_TEST_RESULT'][df_after['ID'] == arr_temp[i]].mean(),2), df_after['AGE'][df_after["ID"] == arr_temp[i]].mean(), df_after['ID'][df_after["ID"] == arr_temp[i]].mean()]
    
print(f"There are {len(after_dict)} observations in the 'after' dataset")




There are 1504 observations in the 'before' dataset
There are 1504 observations in the 'after' dataset


#### Remove NaN values from both before/after datasets

In [9]:
# remove NaN values
before_dict = {k: v for k, v in before_dict.items() if pd.Series(v).notna().all()}

after_dict = {k: v for k, v in after_dict.items() if pd.Series(v).notna().all()}


# Print len of dictionary after removing NaN
print(f"There are {len(before_dict)} observations remaining after removing NaN values in the 'before' dataset")
print(f"There are {len(after_dict)} observations remaining after removing NaN values in the 'after' dataset")


There are 1327 observations remaining after removing NaN values in the 'before' dataset
There are 1126 observations remaining after removing NaN values in the 'after' dataset


### Create new dataframe given cleaned before/after data

In [10]:
df_before_avg = pd.DataFrame(before_dict.items(), columns = ['ID', 'avg_gtt_test_result'])
# df_before_avg['language'] = df_before_avg['avg_gtt_test_result'].str[2]
df_before_avg['id_num'] = df_before_avg['avg_gtt_test_result'].str[2]
df_before_avg['age'] = df_before_avg['avg_gtt_test_result'].str[1]
df_before_avg['avg_before_test'] = df_before_avg['avg_gtt_test_result'].str[0]
df_before_avg["age"] = df_before_avg["age"].astype(int)
df_before_avg = df_before_avg.drop(columns = 'avg_gtt_test_result', axis = 1)


In [11]:
df_after_avg = pd.DataFrame(after_dict.items(), columns = ['ID', 'avg_gtt_test_result'])
# df_after_avg['language'] = df_after_avg['avg_gtt_test_result'].str[2]
df_after_avg['id_num'] = df_after_avg['avg_gtt_test_result'].str[2]
df_after_avg['age'] = df_after_avg['avg_gtt_test_result'].str[1]
df_after_avg['avg_after_test'] = df_after_avg['avg_gtt_test_result'].str[0]
df_after_avg["age"] = df_after_avg["age"].astype(int)
df_after_avg = df_after_avg.drop(columns = 'avg_gtt_test_result', axis = 1)

### Join the resulting before/after dataframes

In [12]:
df_join = df_before_avg.join(df_after_avg, rsuffix='_right')
df_join = df_join.drop(columns = ['ID_right', 'age_right', 'id_num_right'], axis = 1)
df_join['id_num'] = df_join['id_num'].astype(int)



#### check for null values

In [13]:
df_join.isnull().sum()

ID                   0
id_num               0
age                  0
avg_before_test      0
avg_after_test     201
dtype: int64

#### Drop NaN values in the after dataframe, caluclate differnce and percentage change in test results, drop unneeded cols

In [14]:
df_averages = df_join.dropna()
df_averages = df_averages.assign(diff = df_averages['avg_after_test'] - df_averages['avg_before_test'])
df_averages['percent_change'] = round(100*((df_averages['avg_after_test'] - df_averages['avg_before_test'])/df_averages['avg_before_test']),2)

df_averages = df_averages.drop(columns = "ID", axis = 1)
df_averages = df_averages.reset_index()
df_averages = df_averages.rename(columns = {'id_num':'id'})
df_averages = df_averages.drop(columns = 'index', axis = 1)
df_averages


,id,age,avg_before_test,avg_after_test,diff,percent_change
0,2,39,182.0,161.60,-20.40,-11.21
1,3,31,115.5,115.50,0.00,0.00
2,4,42,131.2,136.00,4.80,3.66
3,5,36,110.4,95.40,-15.00,-13.59
4,6,36,65.0,144.00,79.00,121.54
...,...,...,...,...,...,...
1121,1676,25,86.0,103.33,17.33,20.15
1122,1678,30,128.8,144.60,15.80,12.27
1123,1679,24,101.8,140.80,39.00,38.31
1124,1680,22,112.4,164.57,52.17,46.41


### Remove patients who do not have both before and after test results

In [15]:
arr_id = list(df_averages.id.values)

In [16]:
df_clean = df[df.ID.isin(arr_id)]

In [17]:
df_clean

,ID,LANGUAGE,AGE,GTT_TEST_RESULT,High_WNL,Before_After,LMP_DATE,24w_DATE,GTT_TEST_NAME,GTT_RESULT_DATE
5,2,Spanish,39,91,WNL,After,1/18/2020,7/4/2020,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",1/23/2021
6,2,Spanish,39,191,High,After,1/18/2020,7/4/2020,"GLUCOSE, GESTATIONAL SCREEN (50G)",1/14/2021
7,2,Spanish,39,182,High,Before,1/18/2020,7/4/2020,"GLUCOSE, GESTATIONAL SCREEN (50G)",3/13/2020
8,2,Spanish,39,189,High,After,1/18/2020,7/4/2020,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",1/23/2021
9,2,Spanish,39,197,High,After,1/18/2020,7/4/2020,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",1/23/2021
...,...,...,...,...,...,...,...,...,...,...
8166,1681,Spanish,39,131,WNL,Before,6/15/2019,11/30/2019,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",8/28/2019
8167,1681,Spanish,39,157,WNL,Before,6/15/2019,11/30/2019,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",8/28/2019
8168,1681,Spanish,39,148,High,Before,6/15/2019,11/30/2019,"GLUCOSE, GESTATIONAL SCREEN (50G)",8/25/2019
8169,1681,Spanish,39,110,WNL,After,6/15/2019,11/30/2019,"GTT, GESTATIONAL, 3 HR,4 SPEC (100G)",12/5/2019


### Create Pivot Table of Patients who tested "High" or "WNL" before and after 24 weeks by Test Name

In [18]:
table = pd.pivot_table(df_clean, values='ID', index=['High_WNL','GTT_TEST_NAME'], columns='Before_After', aggfunc= 'count')

In [19]:
table

Before_After                                   After  Before
High_WNL GTT_TEST_NAME                                      
High     GLUCOSE, GESTATIONAL SCREEN (50G)       392     663
         GTT, GESTATIONAL, 3 HR,4 SPEC (100G)    388     390
WNL      GLUCOSE, GESTATIONAL SCREEN (50G)       563     978
         GTT, GESTATIONAL, 3 HR,4 SPEC (100G)   1767    2325

### Re-join Language to DF

In [20]:
df_b = df
df_b = df_b.groupby('ID', as_index = False).first()
df_b = df_b.rename(columns = {'ID':'id'})
df_merge = pd.merge(df_averages, df_b, how = 'inner', on = 'id')

In [21]:
df_averages = df_merge.drop(columns = ['AGE', 'GTT_TEST_RESULT', 'High_WNL', 'Before_After', 'LMP_DATE', '24w_DATE', 'GTT_TEST_NAME', 'GTT_RESULT_DATE'], axis = 1 )
df_averages = df_averages.rename(columns = {'LANGUAGE': 'language'})



In [23]:
df_averages

,id,age,avg_before_test,avg_after_test,diff,percent_change,language
0,2,39,182.0,161.60,-20.40,-11.21,Spanish
1,3,31,115.5,115.50,0.00,0.00,Spanish
2,4,42,131.2,136.00,4.80,3.66,Spanish
3,5,36,110.4,95.40,-15.00,-13.59,English
4,6,36,65.0,144.00,79.00,121.54,English
...,...,...,...,...,...,...,...
1121,1676,25,86.0,103.33,17.33,20.15,English
1122,1678,30,128.8,144.60,15.80,12.27,Spanish
1123,1679,24,101.8,140.80,39.00,38.31,Spanish
1124,1680,22,112.4,164.57,52.17,46.41,English


In [24]:
# count_before = 0
# count_after = 0
# patient_arr = []

# for i in range(len(df)):
#     if df[i,0] not in patient_arr:
#         patient_arr.append(df[i,0])
#         if(df.iloc[1,5] == "Before"):
#             count_before

In [25]:
df.iloc[1,0]

1

#### Get Dummy variables for language var and drop all languages except spanish and english

In [26]:
df_averages
df_dummies = pd.get_dummies(df_averages['language'])
del df_dummies[df_dummies.columns[-1]]
df_new = pd.concat([df_averages, df_dummies], axis=1)
del df_new['language']
df_new = df_new.drop(columns = ['American Sign Language', 'Arabic', 'Mixtec', 'Punjabi'], axis = 1)
df_new


,id,age,avg_before_test,avg_after_test,diff,percent_change,English,Spanish
0,2,39,182.0,161.60,-20.40,-11.21,0,1
1,3,31,115.5,115.50,0.00,0.00,0,1
2,4,42,131.2,136.00,4.80,3.66,0,1
3,5,36,110.4,95.40,-15.00,-13.59,1,0
4,6,36,65.0,144.00,79.00,121.54,1,0
...,...,...,...,...,...,...,...,...
1121,1676,25,86.0,103.33,17.33,20.15,1,0
1122,1678,30,128.8,144.60,15.80,12.27,0,1
1123,1679,24,101.8,140.80,39.00,38.31,0,1
1124,1680,22,112.4,164.57,52.17,46.41,1,0


# Correlation Statistics 

### Correlation of age and avg test result before 24 weeks

In [27]:
y_before = df_averages['age']
x_before = df_averages['avg_before_test']

pearson_corr_before = x_before.corr(y_before)
spearman_corr_before = x_before.corr(y_before, method = 'spearman')
kendall_corr_before = x_before.corr(y_before, method = 'kendall')

print(f"The pearson correlation coefficient between \npatient's age and average test GTT test result before 24 weeks is: {round(pearson_corr_before,3)}\n")
print(f"The spearman correlation coefficient between \npatient's age and average test GTT test result before 24 weeks is: {round(spearman_corr_before,3)}\n")
print(f"The spearman correlation coefficient between \npatient's age and average test GTT test result before 24 weeks is: {round(kendall_corr_before,3)}\n")

avg_corr_before = (pearson_corr_before+spearman_corr_before+kendall_corr_before)/3


The pearson correlation coefficient between 
patient's age and average test GTT test result before 24 weeks is: 0.282

The spearman correlation coefficient between 
patient's age and average test GTT test result before 24 weeks is: 0.316

The spearman correlation coefficient between 
patient's age and average test GTT test result before 24 weeks is: 0.218



### Correlation of age and avg test result after 24 weeks

In [28]:
y_after = df_averages['age']
x_after = df_averages['avg_after_test']

pearson_corr_after = x_after.corr(y_after)
spearman_corr_after = x_after.corr(y_after, method = 'spearman')
kendall_corr_after = x_after.corr(y_after, method = 'kendall')

print(f"The pearson correlation coefficient between \npatient's age and average test GTT test result after 24 weeks is: {round(pearson_corr_after,3)}\n")
print(f"The spearman correlation coefficient between \npatient's age and average test GTT test result after 24 weeks is: {round(spearman_corr_after,3)}\n")
print(f"The spearman correlation coefficient between \npatient's age and average test GTT test result after 24 weeks is: {round(kendall_corr_after,3)}\n")

avg_corr_after = (pearson_corr_after+spearman_corr_after+kendall_corr_after)/3


The pearson correlation coefficient between 
patient's age and average test GTT test result after 24 weeks is: -0.006

The spearman correlation coefficient between 
patient's age and average test GTT test result after 24 weeks is: -0.011

The spearman correlation coefficient between 
patient's age and average test GTT test result after 24 weeks is: -0.008



### Correlation of age and percent change in test result after 24 weeks

In [29]:
y_change = df_averages['age']
x_change = df_averages['percent_change']

pearson_corr_change = x_change.corr(y_change)
spearman_corr_change = x_change.corr(y_change, method = 'spearman')
kendall_corr_change = x_change.corr(y_change, method = 'kendall')

print(f"The pearson correlation coefficient between \npatient's age and percentage change after 24 weeks is: {round(pearson_corr_change,3)}\n")
print(f"The spearman correlation coefficient between \npatient's age and percentage change after 24 weeks is: {round(spearman_corr_change,3)}\n")
print(f"The spearman correlation coefficient between \npatient's age and percentage change after 24 weeks is: {round(kendall_corr_change,3)}\n")

avg_corr_change = (pearson_corr_change+spearman_corr_change+kendall_corr_change)/3


The pearson correlation coefficient between 
patient's age and percentage change after 24 weeks is: -0.22

The spearman correlation coefficient between 
patient's age and percentage change after 24 weeks is: -0.242

The spearman correlation coefficient between 
patient's age and percentage change after 24 weeks is: -0.166



### Averaged Correlations - Conclusion

In [30]:
print(f"correlation of age and GTT test result prior to 24 weeks is week and \nis statistically insignificant - with an average correlation of: {round(avg_corr_before,3)}\n") 
print(f"correlation of age and GTT test result after to 24 weeks is very weak and \nis statistically insignificant - with an average correlation of: {round(avg_corr_after,3)}\n")
print(f"correlation of age and average change in GTT test result after 24 weeks is weak \nis statistically insignificant - with an average correlation of: {round(avg_corr_change,3)}\n")



correlation of age and GTT test result prior to 24 weeks is week and 
is statistically insignificant - with an average correlation of: 0.272

correlation of age and GTT test result after to 24 weeks is very weak and 
is statistically insignificant - with an average correlation of: -0.008

correlation of age and average change in GTT test result after 24 weeks is weak 
is statistically insignificant - with an average correlation of: -0.209



## Language correlation with percentage change

In [31]:
y_lang = df_new['Spanish']
x_lang = df_new['percent_change']

pearson_corr_lang = x_change.corr(y_lang)
spearman_corr_lang = x_change.corr(y_lang, method = 'spearman')
kendall_corr_lang = x_change.corr(y_lang, method = 'kendall')

print(f"The pearson correlation coefficient between \nSpanish speaking patients and percentage change after 24 weeks is: {round(pearson_corr_lang,3)}\n")
print(f"The spearman correlation coefficient between \nSpanish speaking patients and percentage change after 24 weeks is: {round(spearman_corr_lang,3)}\n")
print(f"The spearman correlation coefficient between \nSpanish speaking patients and percentage change after 24 weeks is: {round(kendall_corr_lang,3)}\n")

avg_corr_lang = (pearson_corr_lang+spearman_corr_lang+kendall_corr_lang)/3


The pearson correlation coefficient between 
Spanish speaking patients and percentage change after 24 weeks is: -0.119

The spearman correlation coefficient between 
Spanish speaking patients and percentage change after 24 weeks is: -0.124

The spearman correlation coefficient between 
Spanish speaking patients and percentage change after 24 weeks is: -0.101



In [32]:
y_lang = df_new['English']
x_lang = df_new['percent_change']

pearson_corr_lang = x_change.corr(y_lang)
spearman_corr_lang = x_change.corr(y_lang, method = 'spearman')
kendall_corr_lang = x_change.corr(y_lang, method = 'kendall')

print(f"The pearson correlation coefficient between \nEnglish speaking patients and percentage change after 24 weeks is: {round(pearson_corr_lang,3)}\n")
print(f"The spearman correlation coefficient between \nEnglish speaking patients and percentage change after 24 weeks is: {round(spearman_corr_lang,3)}\n")
print(f"The spearman correlation coefficient between \nEnglish speaking patients and percentage change after 24 weeks is: {round(kendall_corr_lang,3)}\n")

avg_corr_lang = (pearson_corr_lang+spearman_corr_lang+kendall_corr_lang)/3


The pearson correlation coefficient between 
English speaking patients and percentage change after 24 weeks is: 0.12

The spearman correlation coefficient between 
English speaking patients and percentage change after 24 weeks is: 0.124

The spearman correlation coefficient between 
English speaking patients and percentage change after 24 weeks is: 0.101



In [33]:
print(f"While neither english nor spanish speaking patients showed significant correlation with percentage change \nin test results after 24 weeks, it is worth noting that enlish speaking patients did have a very weak but postive \ncorrelation with percentage change while spanish speaking patients are indicated as having a very weak negative \ncorrelation in terms of percentage changed after 24 weeks")

While neither english nor spanish speaking patients showed significant correlation with percentage change 
in test results after 24 weeks, it is worth noting that enlish speaking patients did have a very weak but postive 
correlation with percentage change while spanish speaking patients are indicated as having a very weak negative 
correlation in terms of percentage changed after 24 weeks
